# r/CasualConversations Database creation
This Jupyter Notebook is meant to create a database from submissions to r/CasualConversations. It first extracts submissions and then saves them into a special database file such that feature can be created from the submissions. This notebook is part of a project that tries to predict the flair of a submission on CasualConversations.

In [26]:
import praw # Source code : https://github.com/praw-dev/praw
            # helpful page: https://praw.readthedocs.io/en/latest/code_overview/praw_models.html

# Initialising a connection to Reddit
reddit = praw.Reddit(user_agent='Doing some stuff', client_id='hir_Veg3Rs-Svw', client_secret="giRkbKknUe4Oy4EcoGYA1N_Y0sA",
                    username='', password='')

In [81]:
# General modules and function(s)
import numpy as np
from datetime import datetime
import re
import os
import json
import ijson
from tqdm import tqdm
import pandas as pd

def utc_time(timestamp):
    '''Changes Unix to UTC time'''
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

## Filter online JSON database of reddit posts in a certain month
Certain templated flairs exist as of March 2019 in r/CasualConversation. However, in previous months, those templated flairs were different or users used their own flairs. Therefore, the approach is to find all the submissions in the JSON database that are made in r/CasualConversations and filter these submissions based on the flairs that can be recognised a topic of discussion. For example, the submissions that have a flair containing the removal reason of the post or a random flair thought up by a user (e.g. 🍕 This subreddit is now sponsored by Domino's), will be filtered out.

### Set-up

In [27]:
r_CasualConversation = reddit.subreddit('CasualConversation')

In [3]:
if 'all_submission_ids' not in locals():
    all_submissions = []
    all_submission_ids = []
    print('"all_submissions" and "all_submission_ids" have been initialised.')
else:
    print('"all_submissions" and "all_submission_ids" already exist')

"all_submissions" and "all_submission_ids" have been initialised.


### Extracting the submissions that are made in r/CasualConversation

In [96]:
all_submissions = []
all_flairs = []
submissions_found = 0

print('Submissions found :')
with open('RS_2018-10') as json_data:
    for line in json_data:
        json_line = json.loads(line)
        if(json_line['subreddit'] == 'CasualConversation'):
            all_submissions.append(json_line)
            all_flairs.append(json_line['link_flair_text'])
            submissions_found += 1
            if(json_line['link_flair_text'] != None):
                print('Count : ' + str(submissions_found) + ' '*(10-len(str(submissions_found))) +
                      'Title : ' + json_line['title'][:40] + ' '*(50-len(str(json_line['title'][:40]))) +
                      'Flair : ' + json_line['link_flair_text']
                     )
            else:
                print('Count : ' + str(submissions_found) + ' '*(10-len(str(submissions_found))) +
                      'Title : ' + json_line['title'][:40] + ' '*(50-len(str(json_line['title'][:40]))) +
                      'Flair : ' + 'None'
                     )

Submissions found :
Count : 1         Title : The worst part about being an adult is t          Flair : no context⇢
Count : 2         Title : I hate social anxiety and being alone in          Flair : banned topic⇢
Count : 3         Title : wendy from ozark is so god damn gorgeous          Flair : no context⇢
Count : 4         Title : Feeling bad about buying food at the air          Flair : None
Count : 5         Title : New job starts tomorrow!                          Flair : None
Count : 6         Title : Good Weekend after a long time!                   Flair : None
Count : 7         Title : It’s late at night and I’m alone with my          Flair : None
Count : 8         Title : Had a pretty good week yet...                     Flair : None
Count : 9         Title : The fact that ecosystems thrive on balan          Flair : no context⇢
Count : 10        Title : Are the new Macs REALLY that bad?                 Flair : None
Count : 11        Title : Need some "Movie Character Day" co

Count : 90        Title : I'm 22 and have a desire to be adopted!           Flair : None
Count : 91        Title : Which wig should I choose? (link to poll          Flair : no context⇢
Count : 92        Title : I’ve been seeing a lot of posts lately j          Flair : banned topic⇢
Count : 93        Title : I want to try scuba diving but doing som          Flair : abandoned⇢
Count : 94        Title : Anyone here? I feel like talking.                 Flair : None
Count : 95        Title : My fiancé found an abandoned kitten and           Flair : None
Count : 96        Title : How old are you, and how much money do y          Flair : no context⇢
Count : 97        Title : Any ideas for pun based house decoration          Flair : None
Count : 98        Title : Reddit followers                                  Flair : None
Count : 99        Title : Cubicle/office workers, how do you stay           Flair : None
Count : 100       Title : Is “having a charity” some type of incom          Flair

Count : 179       Title : i do not know                                     Flair : None
Count : 180       Title : I need some karma so I can post on this           Flair : no context⇢
Count : 181       Title : Laugh to start your work week: Adam Driv          Flair : no context⇢
Count : 182       Title : Today is my birthday and I didn't even n          Flair : None
Count : 183       Title : Roped my first goat today                         Flair : None
Count : 184       Title : I hate the fact that i dont have anyone           Flair : None
Count : 185       Title : I need a little bit of karma so I can ma          Flair : None
Count : 186       Title : So I have been listening to this pianist          Flair : None
Count : 187       Title : Gushers are so tasty that they've never           Flair : no context⇢
Count : 188       Title : I keep seeing this girl on campus                 Flair : banned topic⇢
Count : 189       Title : I wasn’t sure where to post this or why           Flai

Count : 267       Title : Is there any way to step back?                    Flair : None
Count : 268       Title : Can I have some encouragement/How should          Flair : None
Count : 269       Title : I hope the physical keyboard for cell ph          Flair : None
Count : 270       Title : I cannot motivate myself to do anything           Flair : None
Count : 271       Title : Little scary dream where I am a creep             Flair : None
Count : 272       Title : I could have sworn the actor for Charlie          Flair : no context⇢
Count : 273       Title : My doctor told me there is nothing I can          Flair : None
Count : 274       Title : it’s my birthday!                                 Flair : None
Count : 275       Title : I don't get why the US just doesn't lega          Flair : no context⇢
Count : 276       Title : Would it be weird to go up to a guy, tel          Flair : banned topic⇢
Count : 277       Title : Album recommendations?                            Flair : no 

Count : 356       Title : My Self-Realosation                               Flair : abandoned⇢
Count : 357       Title : Pet owners who got a pet after you reach          Flair : None
Count : 358       Title : Am I a feminist?                                  Flair : banned topic⇢
Count : 359       Title : You ever feel like everyone wants to mak          Flair : None
Count : 360       Title : I constantly think of other people if th          Flair : no context⇢
Count : 361       Title : What are your reviews of medium.com?              Flair : abandoned⇢
Count : 362       Title : Why does some people have some concerns           Flair : abandoned⇢
Count : 363       Title : I slapped a kid in the face today                 Flair : None
Count : 364       Title : I feel like tf2 should have a slot in ma          Flair : no context⇢
Count : 365       Title : I constantly think of other people if th          Flair : no context⇢
Count : 366       Title : Ok i get its odd but i love masks   

...

Count : 2387      Title : A blazing row with the wife because you’          Flair : no context⇢
Count : 2388      Title : Best and cheapest car insurance in Dubai          Flair : banned topic⇢
Count : 2389      Title : I'm pretty sure I'm an alcoholic                  Flair : banned topic⇢
Count : 2390      Title : Would selling “No soliciting” signs door          Flair : no context⇢
Count : 2391      Title : It’s 2:30am and I’m listening to sad mus          Flair : Music
Count : 2392      Title : I don't know what I want to do in life..          Flair : None
Count : 2393      Title : Would you leave somewhere that makes you          Flair : None
Count : 2394      Title : I love carrying back my pudding to my de          Flair : None
Count : 2395      Title : CHANGE CITIZENSHIP/// New Identity//New           Flair : no context⇢
Count : 2396      Title : Hurricane Michael - the strongest hurric          Flair : None
Count : 2397      Title : Watched any good horror movies this mont    

Count : 2475      Title : Anyone else just completely ok with bein          Flair : None
Count : 2476      Title : I haven't been doing any of the work I s          Flair : None
Count : 2477      Title : Anyone also miss r/watchpeopledie?                Flair : banned topic⇢
Count : 2478      Title : Stuck in the bathroom                             Flair : None
Count : 2479      Title : How do you guys wake up early?                    Flair : None
Count : 2480      Title : My husband and my ex-husband are standin          Flair : no context⇢
Count : 2481      Title : I really hate                                     Flair : no context⇢
Count : 2482      Title : So stressed and lonely on days off that           Flair : None
Count : 2483      Title : I keep reading this creepy r/AskReddit p          Flair : None
Count : 2484      Title : Girls who wear heels every day, even to           Flair : None
Count : 2485      Title : I really hate r/GetMotivated                      Flair : Non

Count : 2563      Title : Look for place to advertise my designs            Flair : None
Count : 2564      Title : Bored at work, talk to me about anything          Flair : no context⇢
Count : 2565      Title : Your far off dreams and aspirations?              Flair : None
Count : 2566      Title : Jesus Christ I love cats.                         Flair : None
Count : 2567      Title : I miss Candystand                                 Flair : None
Count : 2568      Title : Feeling pretty down and want some good n          Flair : None
Count : 2569      Title : I love living by the Friday night lights          Flair : abandoned⇢
Count : 2570      Title : I feel guilty for spending my parents mo          Flair : banned topic⇢
Count : 2571      Title : Would you buy a game console purely beca          Flair : Gaming
Count : 2572      Title : Feel like talking about the bible?                Flair : None
Count : 2573      Title : It's my 20th birthday tomorrow (on the 1          Flair : ba

Count : 2652      Title : What’s your favorite album and music gen          Flair : no context⇢
Count : 2653      Title : Do certain smells ever bring back memori          Flair : None
Count : 2654      Title : My dog died                                       Flair : banned topic⇢
Count : 2655      Title : My clients and superior hates me because          Flair : None
Count : 2656      Title : Does anybody here listen to boiler room           Flair : abandoned⇢
Count : 2657      Title : Was in the bathroom stall at work and co          Flair : no context⇢
Count : 2658      Title : Unpaid research intern                            Flair : no context⇢
Count : 2659      Title : I’m confused on whether or not this girl          Flair : banned topic⇢
Count : 2660      Title : Guys and Girls my day has been pretty su          Flair : Music
Count : 2661      Title : Today is my 4 year cake day!                      Flair : banned topic⇢
Count : 2662      Title : Was in the bathroom stall at 

KeyboardInterrupt: 

### Filtering based on flair

In [41]:
flair_templates = list(r_CasualConversation.flair.link_templates)

flair_ids = []
for flair_template in flair_templates:
    flair_ids.append(flair_templates)
    print(flair_template['text'])

:gaming: Gaming
:chat: Just Chatting
:note: Music
:film: Movies & Shows
:food: Food & Drinks
:paw: Pets & Animals
:mark: Books & Reading
:story: Life Stories
:question: Questions
:thinking: Thoughts & Ideas
:ididit: Made did it


In [97]:
all_flairs_set = set(all_flairs)
flair_column = []
count_column = []

for flair in all_flairs_set:
    if(flair != None):
        flair_column.append(flair)
        count_column.append(all_flairs.count(flair))
#         print(flair + ' ' + str(all_flairs.count(flair)))
flair_count_df = pd.DataFrame(data={'flair':flair_column, 'count':count_column})
flair_count_df = flair_count_df.sort_values(by='count',ascending=False)
flair_count_df

,flair,count
16,no context⇢,567
7,banned topic⇢,338
6,abandoned⇢,208
8,Music,41
3,Gaming,21
2,Movies &amp; Shows,17
12,Book,11
11,CAPS⇢,7
1,Neat,6
15,no context,6
